VISION (Planar Coordinate)

In [6]:
from pypylon import pylon
import cv2
import numpy as np
import time
import csv

def save_coordinates_to_csv(coord, timestamp):
    with open("object_coordinates.csv", "a", newline='') as file:
        writer = csv.writer(file)
        writer.writerow([timestamp, *coord])

camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
camera.Open()
camera.Width.SetValue(1600)
camera.Height.SetValue(1200)
camera.PixelFormat.SetValue('BayerRG8')

converter = pylon.ImageFormatConverter()
converter.OutputPixelFormat = pylon.PixelType_BGR8packed
converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned

frame_count = 0
t0 = time.time()
last_saved_time = 0

camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly)
while camera.IsGrabbing():
    grab_result = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

    if grab_result.GrabSucceeded():
        current_time = time.time()
        elapsed_time_since_last_save = current_time - last_saved_time

        if elapsed_time_since_last_save >= 0.01:  # 10 milliseconds
            last_saved_time = current_time

            image = converter.Convert(grab_result)
            img_array = image.GetArray()
            display_img = cv2.resize(img_array, (800, 600))

            hsv = cv2.cvtColor(display_img, cv2.COLOR_BGR2HSV)
            lower_red = np.array([0, 120, 70])
            upper_red = np.array([10, 255, 255])
            mask1 = cv2.inRange(hsv, lower_red, upper_red)

            lower_red = np.array([160, 120, 70])
            upper_red = np.array([180, 255, 255])
            mask2 = cv2.inRange(hsv, lower_red, upper_red)

            mask = mask1 + mask2

            contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            circle_center = None
            for contour in contours:
                approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)
                cv2.drawContours(display_img, [approx], 0, (0, 255, 0), 5)
                x, y, w, h = cv2.boundingRect(contour)

                if h >= 10:  # variable of minimum height for the contour
                    circle_center = (x + w // 2, y + h // 2)
                    cv2.circle(display_img, circle_center, h // 2, (0, 255, 0), 2)
                    save_coordinates_to_csv(circle_center, current_time)  # Save coordinates to CSV

            if circle_center:
                cv2.putText(display_img, f"Coord: {circle_center}", (500, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            frame_count += 1
            fps = frame_count / (current_time - t0)
            cv2.putText(display_img, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow('Live Preview', display_img)

        # q --> break loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    grab_result.Release()

cv2.destroyAllWindows()
camera.Close()


In [ ]:
from pypylon import pylon
import cv2
import numpy as np
import time
import csv

def save_coordinates_to_csv(coord, timestamp):
    with open("object_coordinates.csv", "a", newline='') as file:
        writer = csv.writer(file)
        writer.writerow([timestamp, *coord])

camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
camera.Open()
camera.Width.SetValue(1600)
camera.Height.SetValue(1200)
camera.PixelFormat.SetValue('BayerRG8')

converter = pylon.ImageFormatConverter()
converter.OutputPixelFormat = pylon.PixelType_BGR8packed
converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned

frame_count = 0
t0 = time.time()
last_saved_time = 0

camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly)
# ... [Previous code for setup and camera initialization]

while camera.IsGrabbing():
    grab_result = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

    if grab_result.GrabSucceeded():
        image = converter.Convert(grab_result)
        img_array = image.GetArray()
        display_img = cv2.resize(img_array, (800, 600))

        # Convert to HSV color space
        hsv = cv2.cvtColor(display_img, cv2.COLOR_BGR2HSV)

        # Define HSV range for orange color
        lower_orange = np.array([10, 100, 20])  # Adjust these values
        upper_orange = np.array([25, 255, 255]) # Adjust these values
        mask = cv2.inRange(hsv, lower_orange, upper_orange)

        # Find contours
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)

            # Check if the approximated contour has 4 vertices (possible rectangle)
            if len(approx) == 4:
                cv2.drawContours(display_img, [approx], 0, (0, 255, 0), 5)
                x, y, w, h = cv2.boundingRect(approx)
                rect_center = (x + w // 2, y + h // 2)
                cv2.rectangle(display_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                save_coordinates_to_csv(rect_center, time.time())  # Save coordinates to CSV

        # ... [Rest of the code for display and breaking loop]

            if circle_center:
                cv2.putText(display_img, f"Coord: {circle_center}", (500, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            frame_count += 1
            fps = frame_count / (current_time - t0)
            cv2.putText(display_img, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow('Live Preview', display_img)

        # q --> break loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    grab_result.Release()

cv2.destroyAllWindows()
camera.Close()

In [1]:
from pypylon import pylon
import cv2
import numpy as np
import time

camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
camera.Open()
camera.Width.SetValue(1600)
camera.Height.SetValue(1200)
camera.PixelFormat.SetValue('BayerRG8')

converter = pylon.ImageFormatConverter()
converter.OutputPixelFormat = pylon.PixelType_BGR8packed
converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned

camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly)
grab_result = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

if grab_result.GrabSucceeded():
    # Convert to OpenCV format
    image = converter.Convert(grab_result)
    img_array = image.GetArray()
    display_img = cv2.resize(img_array, (800, 600))

    # Select ROI manually
    bbox = cv2.selectROI('Live Preview', display_img, False)
    tracker = cv2.TrackerKCF_create()  # You can choose another tracker type
    tracker.init(display_img, bbox)

    grab_result.Release()

while camera.IsGrabbing():
    grab_result = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

    if grab_result.GrabSucceeded():
        # Convert to OpenCV format
        image = converter.Convert(grab_result)
        img_array = image.GetArray()
        display_img = cv2.resize(img_array, (800, 600))

        # Update tracker
        ok, bbox = tracker.update(display_img)

        # Draw bounding box
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(display_img, p1, p2, (0, 255, 0), 2, 1)
        else:
            # Tracking failure
            cv2.putText(display_img, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

        # Display result
        cv2.imshow('Live Preview', display_img)

        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        grab_result.Release()

cv2.destroyAllWindows()
camera.Close()

KeyboardInterrupt: 

: 

In [7]:
from pypylon import pylon
import cv2
import numpy as np
import time
import csv

# Function to append coordinates, timestamp, and fps to a CSV file
def append_to_csv(filename, bbox, timestamp, fps):
    with open(filename, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([timestamp, *bbox, fps])

# Open the CSV file and write the header
csv_filename = "object_tracking.csv"
with open(csv_filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['timestamp', 'x', 'y', 'width', 'height', 'fps'])

camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
camera.Open()
camera.Width.SetValue(1600)
camera.Height.SetValue(1200)
camera.PixelFormat.SetValue('BayerRG8')

converter = pylon.ImageFormatConverter()
converter.OutputPixelFormat = pylon.PixelType_BGR8packed
converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned

camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly)
grab_result = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

if grab_result.GrabSucceeded():
    # Convert to OpenCV format
    image = converter.Convert(grab_result)
    img_array = image.GetArray()
    display_img = cv2.resize(img_array, (800, 600))

    # Select ROI manually
    bbox = cv2.selectROI('Live Preview', display_img, False)
    tracker = cv2.TrackerKCF_create()  # You can choose another tracker type
    tracker.init(display_img, bbox)

    grab_result.Release()

# Variables for FPS calculation
frame_count = 0
t0 = time.time()

while camera.IsGrabbing():
    grab_result = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

    if grab_result.GrabSucceeded():
        # Convert to OpenCV format
        image = converter.Convert(grab_result)
        img_array = image.GetArray()
        display_img = cv2.resize(img_array, (800, 600))

        # Update tracker
        ok, bbox = tracker.update(display_img)

        # Calculate FPS
        frame_count += 1
        elapsed_time = time.time() - t0
        fps = frame_count / elapsed_time

        # Draw bounding box and display coordinates and FPS
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(display_img, p1, p2, (0, 255, 0), 2, 1)
            
            # Display coordinates
            text_coords = f"X: {p1[0]}, Y: {p1[1]}, W: {p2[0]-p1[0]}, H: {p2[1]-p1[1]}"
            cv2.putText(display_img, text_coords, (600, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            # Save the coordinates, timestamp, and FPS to the CSV
            current_time = time.time()
            append_to_csv(csv_filename, bbox, current_time, fps)

        # Display FPS
        text_fps = f"FPS: {fps:.2f}"
        cv2.putText(display_img, text_fps, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

        # Display result
        cv2.imshow('Live Preview', display_img)

        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        grab_result.Release()

# Clean up
cv2.destroyAllWindows()
camera.Close()


30.11.2023 (Added timestamp, starts when the object starts moving.)

In [8]:
from pypylon import pylon
import cv2
import numpy as np
import time
import csv

# Function to append coordinates, timestamp, and fps to a CSV file
def append_to_csv(filename, bbox, timestamp, fps):
    with open(filename, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([timestamp, *bbox, fps])

# Open the CSV file and write the header
csv_filename = "object_tracking.csv"
with open(csv_filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['timestamp', 'x', 'y', 'width', 'height', 'fps'])

# Setup the camera
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
camera.Open()
camera.Width.SetValue(1600)
camera.Height.SetValue(1200)
camera.PixelFormat.SetValue('BayerRG8')

# Setup image format converter
converter = pylon.ImageFormatConverter()
converter.OutputPixelFormat = pylon.PixelType_BGR8packed
converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned

# Start grabbing from the camera
camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly)
grab_result = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

if grab_result.GrabSucceeded():
    # Convert to OpenCV format
    image = converter.Convert(grab_result)
    img_array = image.GetArray()
    display_img = cv2.resize(img_array, (800, 600))

    # Select ROI manually and start the timer
    bbox = cv2.selectROI('Live Preview', display_img, False)
    start_time = time.time()  # Start time is set here
    tracker = cv2.TrackerKCF_create()  # You can choose another tracker type
    tracker.init(display_img, bbox)

    grab_result.Release()

# Variables for FPS calculation
frame_count = 0
t0 = time.time()

while camera.IsGrabbing():
    grab_result = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

    if grab_result.GrabSucceeded():
        # Convert to OpenCV format
        image = converter.Convert(grab_result)
        img_array = image.GetArray()
        display_img = cv2.resize(img_array, (800, 600))

        # Update tracker
        ok, bbox = tracker.update(display_img)

        # Calculate FPS
        frame_count += 1
        elapsed_time = time.time() - t0
        fps = frame_count / elapsed_time

        # Draw bounding box and display coordinates and FPS
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(display_img, p1, p2, (0, 255, 0), 2, 1)
            
            # Display coordinates
            text_coords = f"X: {p1[0]}, Y: {p1[1]}, W: {p2[0]-p1[0]}, H: {p2[1]-p1[1]}"
            cv2.putText(display_img, text_coords, (600, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            # Calculate relative timestamp
            current_time = time.time()
            relative_timestamp = current_time - start_time

            # Save the coordinates, timestamp, and FPS to the CSV
            append_to_csv(csv_filename, bbox, relative_timestamp, fps)

        # Display FPS
        text_fps = f"FPS: {fps:.2f}"
        cv2.putText(display_img, text_fps, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

        # Display result
        cv2.imshow('Live Preview', display_img)

        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        grab_result.Release()

# Clean up
cv2.destroyAllWindows()
camera.Close()

PermissionError: [Errno 13] Permission denied: 'object_tracking.csv'

In [7]:
from pypylon import pylon
import cv2
import numpy as np
import time
import csv
from datetime import datetime

# Function to generate a unique filename based on the current timestamp
def generate_unique_filename():
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    return f"object_tracking_{current_time}.csv"

# Function to append coordinates, timestamp, and fps to a CSV file
def append_to_csv(filename, bbox, timestamp, fps):
    with open(filename, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([timestamp, *bbox, fps])

# Generate a unique filename for this session
csv_filename = generate_unique_filename()

# Create the new CSV file and write the header
with open(csv_filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['timestamp', 'x', 'y', 'width', 'height', 'fps'])

# Setup the camera
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
camera.Open()
camera.Width.SetValue(1600)
camera.Height.SetValue(1200)
camera.PixelFormat.SetValue('BayerRG8')

# Setup image format converter
converter = pylon.ImageFormatConverter()
converter.OutputPixelFormat = pylon.PixelType_BGR8packed
converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned

# Start grabbing from the camera
camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly)
grab_result = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

if grab_result.GrabSucceeded():
    # Convert to OpenCV format
    image = converter.Convert(grab_result)
    img_array = image.GetArray()
    display_img = cv2.resize(img_array, (800, 600))

    # Select ROI manually and start the timer
    bbox = cv2.selectROI('Live Preview', display_img, False)
    start_time = time.time()  # Start time is set here
    tracker = cv2.TrackerMOSSE_create()  # TrackerMOSSE_create
    tracker.init(display_img, bbox)

    grab_result.Release()

# Variables for FPS calculation
frame_count = 0
t0 = time.time()

while camera.IsGrabbing():
    grab_result = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

    if grab_result.GrabSucceeded():
        # Convert to OpenCV format
        image = converter.Convert(grab_result)
        img_array = image.GetArray()
        display_img = cv2.resize(img_array, (800, 600))

        # Update tracker
        ok, bbox = tracker.update(display_img)

        # Calculate FPS
        frame_count += 1
        elapsed_time = time.time() - t0
        fps = frame_count / elapsed_time

        # Draw bounding box and display coordinates and FPS
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(display_img, p1, p2, (0, 255, 0), 2, 1)
            
            # Display coordinates
            text_coords = f"X: {p1[0]}, Y: {p1[1]}, W: {p2[0]-p1[0]}, H: {p2[1]-p1[1]}"
            cv2.putText(display_img, text_coords, (600, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            # Calculate relative timestamp
            current_time = time.time()
            relative_timestamp = current_time - start_time

            # Save the coordinates, timestamp, and FPS to the CSV
            append_to_csv(csv_filename, bbox, relative_timestamp, fps)

        # Display FPS
        text_fps = f"FPS: {fps:.2f}"
        cv2.putText(display_img, text_fps, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

        # Display result
        cv2.imshow('Live Preview', display_img)

        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        grab_result.Release()

# Clean up
cv2.destroyAllWindows()
camera.Close()

RuntimeException: No device is available or no device contains the provided device info properties. : RuntimeException thrown (file 'TlFactory.cpp', line 694)